# Libraries

In [20]:
import pandas as pd
from getpass import getuser
from collections import defaultdict
from datetime import datetime, timedelta

# Load and inspect dataset

In [21]:
# Get the current user's name
user = getuser()

# Construct the path using the user's name
function_path = f"C:/Users/{user}/Documents/GitHub/tiebreak_wc/code/wiki/functions_uefa.ipynb"

# Run the notebook
%run $function_path

In [22]:


# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\in\eu_goals_women.xlsx'

# Load the Excel file into a DataFrame
df = pd.read_excel(data_path)

# Filter the DataFrame to include only observations where year >= 1997
df = df[df['year'] >= 1997]

# Display the first few rows of the DataFrame to verify the import
display(df.head())

,stage,year,time,home_team,away_team,score,stadium_name,stadium_city,stadium_attendance,referee_name,...,goal_minute,extra_time,goals_home,goals_away,own_goal,penalty,goal_minute_et,goal_et,short_date,long_date
83,Group A,1997,14:00,France,Spain,1–1,Nobelstadion,Karlskoga,920,Denmark,...,50,0,1,1,0,0,0,0,NaN,29 June 1997
84,Group A,1997,14:00,France,Spain,1–1,Nobelstadion,Karlskoga,920,Denmark,...,14,0,1,1,0,0,0,0,NaN,29 June 1997
85,Group A,1997,19:00,Sweden,Russia,2–1,Tingvalla IP,Karlstad,3829,Germany,...,10,0,2,1,0,0,0,0,NaN,29 June 1997
86,Group A,1997,19:00,Sweden,Russia,2–1,Tingvalla IP,Karlstad,3829,Germany,...,82,0,2,1,0,0,0,0,NaN,29 June 1997
87,Group A,1997,19:00,Sweden,Russia,2–1,Tingvalla IP,Karlstad,3829,Germany,...,80,0,2,1,0,0,0,0,NaN,29 June 1997


# Clean, transfrom, create variables

## time

In [23]:
# Step 1: Clean time variable by removing any letters (e.g., EEST, PST) if they exist
df['time_cleaned'] = df['time'].str.replace(r'[A-Za-z]+', '', regex=True).str.strip()

# Step 2: Split time where there are parentheses into local time and UTC offset
df['local_time'] = df['time_cleaned'].str.extract(r'(\d{2}:\d{2})')  # extract the part before parentheses
df['utc_time_offset'] = df['time_cleaned'].str.extract(r'\((.*?)\)')  # extract the part inside parentheses


## Date

In [24]:
# Remove comma before the year in 'long_date' column
df['long_date'] = df['long_date'].str.replace(r',\s*(\d{4})', r' \1', regex=True)

# Convert 'long_date' to datetime format and extract the date in "YYYY-MM-DD" format for missing 'short_date' values
df['short_date'] = df['short_date'].fillna(pd.to_datetime(df['long_date'], format="%d %B %Y").dt.strftime('%Y-%m-%d'))


# Extract relevant columns

In [25]:
# Filter out the knockout stages
df = df[~df['stage'].isin(['Quarter-finals', 'Round of 16', 'Semi-finals', 'Final', 'Knockout stage'])]

# Extract relevant columns for goal events and match results
goals_df = df[['year', 'stage', 'home_team', 'away_team', 'scorer_nationality', 'goal_minute', 'short_date','local_time', 'score']]

# Remove duplicates
goals_df = goals_df.drop_duplicates()

display(goals_df.head())


,year,stage,home_team,away_team,scorer_nationality,goal_minute,short_date,local_time,score
83,1997,Group A,France,Spain,France,50,1997-06-29,14:00,1–1
84,1997,Group A,France,Spain,Spain,14,1997-06-29,14:00,1–1
85,1997,Group A,Sweden,Russia,Sweden,10,1997-06-29,19:00,2–1
86,1997,Group A,Sweden,Russia,Sweden,82,1997-06-29,19:00,2–1
87,1997,Group A,Sweden,Russia,Russia,80,1997-06-29,19:00,2–1


# Recreate Leauge Table after first two matchdays

In [26]:

# Assuming goals_df is your original dataset
agg_goals_before_last_day, goals_last_day_sorted = process_goals_data(goals_df)


In [27]:
agg_goals_before_last_day

,year,stage,home_team,away_team,local_time,short_date,goals_home,goals_away,original_score,calculated_score,score_match,won
0,1997,Group A,France,Spain,14:00,2024-11-08,1,1,1–1,1-1,True,0
1,1997,Group A,Russia,France,19:00,2024-11-08,1,3,1–3,1-3,True,-1
2,1997,Group A,Spain,Sweden,19:00,2024-11-08,0,1,0–1,0-1,True,-1
3,1997,Group A,Sweden,Russia,19:00,2024-11-08,2,1,2–1,2-1,True,1
4,1997,Group B,Denmark,Norway,20:00,2024-11-08,0,5,0–5,0-5,True,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
70,2022,Group C,Sweden,Switzerland,17:00,2024-11-08,2,1,2–1,2-1,True,1
71,2022,Group D,Belgium,Iceland,17:00,2024-11-08,1,1,1–1,1-1,True,0
72,2022,Group D,France,Belgium,20:00,2024-11-08,2,1,2–1,2-1,True,1
73,2022,Group D,France,Italy,20:00,2024-11-08,5,1,5–1,5-1,True,1


In [28]:
home_games, away_games = aggregate_home_away_points(agg_goals_before_last_day)

In [29]:
# Filter for year == 2017 and stage == 'Group D'
home2017d = home_games[(home_games['year'] == 2017) & (home_games['stage'] == 'Group D')]
home2017d

,year,stage,home_team,goals_scored,goals_conceded,points_home,match_count_home
44,2017,Group D,England,8,0,6,2
45,2017,Group D,Scotland,1,2,0,1
46,2017,Group D,Spain,2,0,3,1


In [30]:
# Filter for year == 2021 and stage == 'Group F'
away2017d = away_games[(away_games['year'] == 2017) & (away_games['stage'] == 'Group D')]
away2017d

,year,stage,away_team,goals_scored,goals_conceded,points_away,match_count_away
43,2017,Group D,Portugal,2,3,3,2
44,2017,Group D,Scotland,0,6,0,1
45,2017,Group D,Spain,0,2,0,1


## aggregate data after first two matches 

In [31]:
# Applying the fifa_before_last function to the mock data
all_games_before_last = uefa_before_last(home_games, away_games, agg_goals_before_last_day)

In [32]:
# Filtering all_games_before_last for the year 2017 and stage 'Group D'
group2017d = all_games_before_last[
    (all_games_before_last['year'] == 2017) & 
    (all_games_before_last['stage'] == 'Group D')
]

display(group2017d)

,year,stage,team,goals_scored,goals_conceded,points,goals_difference,total_matches,tiebreaker,tie_won,standing
60,2017,Group D,England,8,0,6,8,2,no need,0,1
61,2017,Group D,Portugal,2,3,3,-1,2,Portugal,1,2
62,2017,Group D,Spain,2,2,3,0,2,Portugal,0,3
63,2017,Group D,Scotland,1,8,0,-7,2,no need,0,4


# Recreate league table after last match day

### uefa criteria 

In [33]:
# Initialize an empty list to store the results for each pair
all_results = []

# Get unique pairs of year, stage, and men
unique_pairs = all_games_before_last[['year', 'stage']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    
    # Apply the function to the current pair
    result = uefa_final_euro(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_results.append(result)

# Concatenate all the results into a single DataFrame
changes_df_euro = pd.concat(all_results)

# Keep only the specified columns
changes_df_euro = changes_df_euro[[ 'year', 'stage', 'team', '1st', '2nd', '3rd', '4th', 'changes']]



=== Initial Standings for Year 1997, Group A Before Last Match Goals ===

  team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_points  before_last_game_standing
Sweden             7                   3                     1                      2                        6                          1
France             5                   4                     2                      2                        4                          2
 Spain             2                   1                     2                     -1                        1                          3
Russia             1                   2                     5                     -3                        0                          4


Analyzing goal: 18 minute, Player team: Sweden, Home: Sweden, Away: France

=== Updated Standings After This Goal ===

  team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  last_game_points  last_game

In [34]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\women\uefa\eu\tb_eu_uefa_women.xlsx'
changes_df_euro.to_excel(file_path, index=False)


# group composition tracking

In [35]:
# Initialize an empty list to store the results for each pair
all_composition_changes = []

# Get unique pairs of year, stage, and men
unique_pairs = all_games_before_last[['year', 'stage']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    year = row['year']
    stage = row['stage']
    
    # Apply the function to the current pair
    composition_changes_df = track_composition_changes(year, stage, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_composition_changes.append(composition_changes_df)

# Concatenate all the results into a single DataFrame
final_composition_changes_df = pd.concat(all_composition_changes, ignore_index=True)

# Display the final DataFrame
display(final_composition_changes_df)




=== Initial Standings for Group A, 1997 (with 0-0 points added) ===
  team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  before_last_game_standing
Sweden             7                   3                     1                      2                          1
France             5                   4                     2                      2                          2
 Spain             2                   1                     2                     -1                          3
Russia             1                   2                     5                     -3                          4



=== Standings after goal at minute 18 in Group A, edition 1997 ===
  team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  last_game_standing  tied_won
Sweden             9                   4                     1                      3                   1         0
France             4                   4                     3  

,year,stage,change_num,goal_time,home_team,away_team,scorer_team,new_top_teams,1st,2nd,3rd,scorer_nationality
0,1997,Group A,0,initial,None,None,NaN,"[Sweden, France]",Sweden,France,Spain,NaN
1,1997,Group A,1,67,Russia,Spain,NaN,"[Sweden, Spain]",Sweden,Spain,France,Spain
2,1997,Group B,0,initial,None,None,NaN,"[Italy, Norway]",Norway,Italy,Germany,NaN
3,1997,Group B,1,82,Denmark,Germany,NaN,"[Germany, Italy]",Italy,Germany,Norway,Germany
4,2001,Group A,0,initial,None,None,None,"[Sweden, Germany]",Germany,Sweden,England,NaN
5,2001,Group B,0,initial,None,None,NaN,"[Italy, Norway]",Norway,Italy,Denmark,NaN
6,2001,Group B,1,37,France,Italy,NaN,"[Denmark, Norway]",Norway,Denmark,Italy,France
7,2005,Group A,0,initial,None,None,NaN,"[Denmark, England]",Denmark,England,Sweden,NaN
8,2005,Group A,1,3,England,Sweden,NaN,"[Denmark, Sweden]",Denmark,Sweden,England,Sweden
9,2005,Group A,2,6,Finland,Denmark,NaN,"[Sweden, Finland]",Sweden,Finland,Denmark,Finland


In [36]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\women\uefa\eu\standings_eu_uefa_women.xlsx'
final_composition_changes_df.to_excel(file_path, index=False)


# best four third placed

In [37]:
# Call the function to get the DataFrame
final_df = best_two_third_placed_eu_women(goals_last_day_sorted, all_games_before_last, agg_goals_before_last_day)

# Define the file path and save to Excel
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\out\wiki\women\uefa\eu\third_teams_eu_uefa_women.xlsx'
final_df.to_excel(file_path, index=False)



--- Processing Year: 2009 ---

=== Initial Standings for Year 2009 Before Processing Any Goals ===

       team   stage  before_last_game_points  before_last_game_goals_scored  before_last_game_goals_conceded  total_goal_difference
    Finland Group A                        6                              3                                1                      2
Netherlands Group A                        3                              3                                2                      1
    Denmark Group A                        3                              2                                2                      0
    Ukraine Group A                        0                              1                                4                     -3
    Germany Group B                        6                              9                                1                      8
     France Group B                        3                              4                                

In [38]:
# Count the number of observations where the year is 2009
count_2009 = goals_last_day_sorted[goals_last_day_sorted['year'] == 2009].shape[0]
print(f"Number of observations in goals_last_day_sorted for the year 2009: {count_2009}")

# Count the number of observations where the year is 2013
count_2013 = goals_last_day_sorted[goals_last_day_sorted['year'] == 2013].shape[0]
print(f"Number of observations in goals_last_day_sorted for the year 2013: {count_2013}")


Number of observations in goals_last_day_sorted for the year 2009: 11
Number of observations in goals_last_day_sorted for the year 2013: 13
